##### Import necessary ibraries

In [1]:
from utils import pickle_to, pickle_from, ignore_warnings
import pandas as pd
import pickle
import seaborn as sns
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
pd.set_option('display.max_colwidth', -1)


# Scrub data
###### Encoding newly created variable names into 0 or 1

Encoding | Label
-- | --
0 | FAKE
1 | REAL



In [2]:
def encode_label(label):
    """Encode real as 1, fake as 0, everything else as None"""
    if label == "REAL":
        return 1
    elif label == "FAKE":
        return 0
    else:
        return None

In [3]:
# Load raw data
raw = pickle_from('../data/raw/raw.pkl')



Loaded file from ../data/raw/raw.pkl


In [6]:
# Apply label encoding function to raw data

raw['numeric_label'] = raw['label'].apply(encode_label)
raw.head()

Unnamed: 0  \
0  8476         
1  10294        
2  3608         
3  10142        
4  875          

                                                                                   title  \
0  You Can Smell Hillary’s Fear                                                            
1  Watch The Exact Moment Paul Ryan Committed Political Suicide At A Trump Rally (VIDEO)   
2  Kerry to go to Paris in gesture of sympathy                                             
3  Bernie supporters on Twitter erupt in anger against the DNC: 'We tried to warn you!'    
4  The Battle of New York: Why This Primary Matters                                        

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   

In [8]:
# raw[raw.text.str.contains('turtle')]

In [9]:
# Removing rows that have the string 'sweeping consequences'
# These have been observed to be duplicates of a single line

raw = raw[~raw['text'].str.contains('sweeping consequence')]




In [10]:
# Removing rows that have the string 'automatically' 
# as this is part of html code

raw = raw[~raw['text'].str.contains('automatically')]
raw = raw.drop(220).reset_index()
raw = raw.drop('index',axis = 1)





In [45]:
# Dropping null values

interim_data = raw[['title','text','numeric_label']].dropna()


In [46]:
interim_data.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 6231 entries, 0 to 6230
Data columns (total 3 columns):
title            6231 non-null object
text             6231 non-null object
numeric_label    6231 non-null int64
dtypes: int64(1), object(2)
memory usage: 194.7+ KB


##### Combining text and title data into a single column by name 'news'

In [47]:
interim_data['news'] = interim_data['title'] + '. '  + interim_data['text']

In [48]:
interim_data.head()

title  \
0  You Can Smell Hillary’s Fear                                                            
1  Watch The Exact Moment Paul Ryan Committed Political Suicide At A Trump Rally (VIDEO)   
2  Kerry to go to Paris in gesture of sympathy                                             
3  Bernie supporters on Twitter erupt in anger against the DNC: 'We tried to warn you!'    
4  The Battle of New York: Why This Primary Matters                                        

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          

## Cleaning text data

Involves the following <br>

* converting everyting to lowercase
* removing punctuations
* removing numbers
* removing non english words


In [65]:
interim_token = interim_data.copy()


# Converting to lowercase
interim_token.text = interim_token.text.apply(lambda x:x.lower())
interim_token['text'] = interim_token['text'].str.replace("’","'")


#Removing all punctuations
interim_token.text = interim_token.text.str.replace('[^\w\s]','')

# Removing numbers
interim_token.text = interim_token.text.str.replace('\d+', ' ')

# Making sure any double-spaces are single
interim_token.text = interim_token.text.str.replace('  ',' ')


In [66]:
# Looking at a sample of cleaned text data 

interim_token.sample(5)



title  \
5065  CAN IT GET MORE CORRUPT? Bill That Bans Naming Officers Involved in Shootings Goes to Pennsylvania Governor   
1320  Magnificent Jellyfish Gardens Purify The Water While Growing Food                                             
1870  Anti-War Movement Anticipates More War Under A Clinton Presidency                                             
610   How To Reduce Stress                                                                                          
4014  Clinton commits Benghazi gaffe, saying US 'didn't lose a single person' in Libya                              

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             

In [67]:
# Choose only columns - label and text

interim_token = interim_token[['numeric_label','text']].copy()



In [68]:
interim_token.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6231 entries, 0 to 6230
Data columns (total 2 columns):
numeric_label    6231 non-null int64
text             6231 non-null object
dtypes: int64(1), object(1)
memory usage: 146.0+ KB


In [69]:
interim_token.head(1)

numeric_label  \
0  0               

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          

### Tokenize & Lemmatize

In [70]:
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()

def lemmatize_text(text):
    return [lemmatizer.lemmatize(w) for w in w_tokenizer.tokenize(text)]



In [71]:
interim_token['tokenized'] = interim_token.text.apply(lemmatize_text)


In [72]:
interim_token.head(1)


numeric_label  \
0  0               

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          

### Remove english stopwords

In [73]:
stop_words = stopwords.words('english')


In [74]:
# Customizing stop words list 

stop_words.append('tweet')
stop_words.append('home')
stop_words.append('headlines')
stop_words.append('finance')
stop_words.append('news')
stop_words.append('was')
stop_words.append('has')
stop_words.append('said')
stop_words.append('wa')
stop_words.append('ha')
stop_words.append('leave')
stop_words.append('comment')
stop_words.append('loading')
stop_words.append('eaten')
stop_words.append('matrix')
stop_words.append('extraterrestrial')
stop_words.append('wi')
stop_words.append('ivt')



In [75]:
interim_token['tokenized'] = interim_token['tokenized'].apply(lambda x: [item for item in x if item not in stop_words])

# Remove words that have less than 3 characters
interim_token['text'] = interim_token['text'].str.replace(r'\b(\w{1,2})\b', ' ')

In [76]:
# Check if text has phrases that are part of html code
interim_token[interim_token['text'].str.contains('phrase block')]


,numeric_label,text,tokenized


In [77]:
# Check if text has phrases that are part of html code
interim_token[interim_token.text.str.contains('automatically')].head()


,numeric_label,text,tokenized


In [78]:
interim_token.head()


numeric_label  \
0  0               
1  0               
2  1               
3  0               
4  1               

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          

### Pickling

In [79]:
# Pickling

pickle_to(interim_token,'../data/interim/interim_token.pkl')
pickle_to(interim_data,'../data/interim/interim_data.pkl')



Sucessfully saved to ../data/interim/interim_token.pkl
Sucessfully saved to ../data/interim/interim_data.pkl
